 Andrew J Markland, ajm259@uakron.edu\
 The University of Akron, CEPS, School of Computer Science\
 Applied Machine Learning CPSC-436-010\
 Dr. Zhong-Hui Duan\

# Project 1, kNN
 -Project Scope:\
     Apply the machine learning classification algorithm "k nearest neighbors"(kNN) to correctly classify a dataset of people based on their risk for MI, Myocardial Infarction.  

In [12]:
#imports, decided to keep that at the top to keep them global 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix

In [28]:
people_df = pd.read_csv("NHANES_data_train.csv")
#partitioning random test data and and training data
trainDf, testDf = train_test_split(people_df, test_size = 0.2, random_state = None)

#Get counts of persons with MI and noMI
MIcount = (trainDf["MI"] == 1).sum()
noMIcount = (trainDf["MI"] == 2).sum()
#Isolate the same number of NoMI's as the MI subgroup
MI_sample = trainDf[trainDf["MI"] == 1]
NoMI_sample = (trainDf[trainDf["MI"] == 2]).sample(frac=(MIcount/noMIcount))
#fTrainSample is the final sample group to train the KNN model on, comprising the same number of
     #people with MI's and people without MI's and on a random scale
fTrainSample = (pd.concat([MI_sample, NoMI_sample])).sample(frac=1)





      ParticipantID  Income  Sex  Age  Race  Edu  Diastolic    Systolic  \
665             666    5.00    1   55     3  NaN  82.333333  125.333333   
2001           2002     NaN    1   61     2  5.0  96.000000  158.000000   
2872           2873    3.14    1   58     3  3.0  66.666667  106.000000   
3544           3545    4.64    2   56     3  NaN  84.333333  130.000000   
3380           3381    1.96    2   48     4  NaN  78.666667  120.000000   
...             ...     ...  ...  ...   ...  ...        ...         ...   
3629           3630    1.88    1   66     7  4.0  78.000000  146.666667   
2201           2202    0.47    1   73     1  1.0  67.333333  174.000000   
2601           2602    5.00    2   50     3  NaN  84.333333  122.666667   
2322           2323    2.65    1   61     3  NaN  79.666667  136.333333   
2607           2608     NaN    1   51     2  NaN  84.666667  135.000000   

          Pulse   BMI  HDL  Trig  LDL  TCHOL  kidneys_eGFR  Diabetes  \
665   74.000000  41.3   30 